# ENTRENAMIENTO MODELO CHURN
----

## Librerías

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from scipy.stats import chi2_contingency
from sklearn.metrics import auc, roc_curve

## Funciones

In [ ]:
def chi_square(df, target, input_cols, threshold):

    statistical_significance=[]

    for attr in input_cols:
        data_count=pd.crosstab(df[attr],df[target])
        obs=data_count.values
   
        chi2, p, dof, expected = chi2_contingency(obs)

        statistical_significance.append([attr,np.round(p,4)])

    statistical_significance=pd.DataFrame(statistical_significance)
    statistical_significance.columns=["Attribute","P-value"]

    df_mod_cols = statistical_significance[statistical_significance["P-value"]<threshold].Attribute.tolist() 
  
    return df_mod_cols, statistical_significance

In [ ]:
def predict_and_get_auc(model, X_train, X_test, y_train, y_test):
    
    y_train_prob = model.predict_proba(X_train)
    y_test_prob = model.predict_proba(X_test)

    fpr, tpr, threshold = roc_curve(y_train, y_train_prob[:, 1])
    print("AUC train = ", round(auc(fpr, tpr), 2))

    fpr, tpr, threshold = roc_curve(y_test, y_test_prob[:, 1])
    print("AUC test = ", round(auc(fpr, tpr), 2))

# 1. Tablón entrenamiento

### Construcción dataset con variables input

In [ ]:
# Cargamos datasets de diciembre

clientes_diciembre_df = pd.read_csv("datos/clientes_2023-12-01.csv", sep='|')
consumos_diciembre_df = pd.read_csv("datos/consumos_2023-12-01.csv", sep='|')
financiacion_diciembre_df = pd.read_csv("datos/financiacion_2023-12-01.csv",sep='|')
productos_diciembre_df = pd.read_csv("datos/productos_2023-12-01.csv", sep= '|')

In [ ]:
# Unimos datasets de diciembre

df_diciembre = clientes_diciembre_df.merge(consumos_diciembre_df, on="id", how="left")
df_diciembre = df_diciembre.merge(financiacion_diciembre_df, on="id", how="left")
df_diciembre = df_diciembre.merge(productos_diciembre_df, on="id", how="left")

### Construcción de la columna target

In [ ]:
# Cargamos clientes de enero

df_enero = pd.read_csv("datos/clientes_2024-01-01.csv", sep= '|')

df_enero['target'] = 0
df_enero = df_enero[['id','target']]

# Hacemos left join de enero sobre diciembre, para que así aparezcan todos los clientes del dataset de diciembre
# Imputamos los NA de target con 1 (clientes que estaban en diciembre pero no en enero -> se han ido de la compañia)

df = pd.merge(df_diciembre, df_enero, on = 'id', how='left')
df = df.fillna({'target':1})

In [ ]:
df = df.drop('id', axis = 1) # las columnas identificadoras no sirven para el modelo

### Corrección de inconsistencias

In [ ]:
df2 = df.copy() # creamos una copia para no sobrescribir el df original

In [ ]:
df2.loc[df2["vel_conexion"] < 0, 'vel_conexion'] = np.nan
df2.loc[df2['conexion']=='adsl', 'conexion'] = 'ADSL'

### Tratamiento de nulos

In [ ]:
df2 = df2.fillna({'descuentos':'NO', 'financiacion':'NO', 'incidencia':'NO'})
df2 = df2.fillna({'num_dt':0, 'imp_financ':0})
df2 = df2.fillna({'vel_conexion':df2['vel_conexion'].mean()})

### Conversión de categóricas a numéricas

In [ ]:
df2['financiacion'] = np.where(df2['financiacion']=="SI", 1, 0)
df2['incidencia'] = np.where(df2['incidencia']=="SI", 1, 0)
df2['descuentos'] = np.where(df2['descuentos']=="SI", 1, 0)

### Ingeniería de características

In [ ]:
año_actual = datetime.now().year
df2['antiguedad'] = año_actual - df2['antiguedad']

### Selección previa

In [ ]:
y = df2["target"]
X = df2.drop(columns=["target"])

In [ ]:
var_significant, statistical_significance = chi_square(df2, 'target',  X.columns, 0.05)

In [ ]:
X = X[var_significant]

### División en train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 0)

### Transformadores

In [ ]:
# Obtenemos lista con los nombres de las columnas categóricas y numéricas por separado

cat_cols = X.select_dtypes(include = ['object']).columns
num_cols = X.select_dtypes(include = ['integer', 'float']).columns

In [ ]:
# Definir transformadores para columnas categóricas y numéricas

transformadores = [
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop = "first"), 
     cat_cols),  # OneHotEncoder para las columnas categoricas
    ('scaler', StandardScaler(), num_cols)]  # StandardScaler para las columnas numéricas

# ColumnTransformer facilita aplicar diferentes transformaciones segun la columna
preprocesador = ColumnTransformer(transformadores)

# Crear el pipeline completo con OneHotEncoder seguido de StandardScaler
pipeline = Pipeline(steps=[
    ('preprocesador', preprocesador)])

In [ ]:
# Ajustar el pipeline sobre el conjunto de datos de train y transformar los datos de train y test

X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

# 2. Entrenamiento del modelo

In [ ]:
params = {"max_depth": [5, 6, 7, 8],
          "min_samples_leaf": [10, 15, 20]}

rf = RandomForestClassifier()

rf_cv = GridSearchCV(rf, params, cv=3, scoring = 'roc_auc')

rf_cv.fit(X_train,y_train)

rf_cv.best_params_

In [ ]:
rf =  RandomForestClassifier(**rf_cv.best_params_, random_state = 0)

rf.fit(X_train, y_train)

In [ ]:
predict_and_get_auc(rf, X_train, X_test, y_train, y_test)